In [1]:
%load_ext lab_black

In [2]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier

In [3]:
data = pd.read_csv("census_cleaned.csv")

In [4]:
data

,age,workclass,fnlgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [5]:
target_feature_name = "salary"

categorical_feature_names = [
    "workclass",
    "education",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "native-country",
]

numeric_feature_names = [
    "age",
    "fnlgt",
    "education-num",
    "capital-gain",
    "capital-loss",
    "hours-per-week",
]

In [6]:
Y = data.pop("salary")
X = data

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20)

In [8]:
numeric_transformer = make_pipeline(SimpleImputer(), StandardScaler())
categorical_transformer = make_pipeline(OneHotEncoder())

preprocessor = ColumnTransformer(
    transformers=[
        ("numeric", numeric_transformer, numeric_feature_names),
        ("categorical", categorical_transformer, categorical_feature_names),
    ],
    remainder="drop",
)

pipe = Pipeline(
    steps=[("preprocessor", preprocessor), ("random_forest", RandomForestClassifier()),]
)

In [9]:
pipe.fit(X_train, Y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['age', 'fnlgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('categorical',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEn

In [10]:
pipe.predict_proba(X_test)

ValueError: Found unknown categories ['Holand-Netherlands'] in column 7 during transform

In [ ]:
for x in X_test.columns:
    print(x.replace("-", "_"))

In [ ]:
with open("../config.yml") as fp:
    config = yaml.load(fp, Loader=yaml.FullLoader)

In [ ]:
config["modeling"]

In [16]:
data.sample(1).to_json(orient="index")

'{"19668":{"age":36,"workclass":"Private","fnlgt":184655,"education":"HS-grad","education-num":9,"marital-status":"Married-civ-spouse","occupation":"Machine-op-inspct","relationship":"Husband","race":"White","sex":"Male","capital-gain":0,"capital-loss":0,"hours-per-week":48,"native-country":"United-States"}}'

In [14]:
import requests

In [17]:
request_data = {
    "age": 36,
    "workclass": "Private",
    "fnlgt": 184655,
    "education": "HS-grad",
    "education-num": 9,
    "marital-status": "Married-civ-spouse",
    "occupation": "Machine-op-inspct",
    "relationship": "Husband",
    "race": "White",
    "sex": "Male",
    "capital-gain": 0,
    "capital-loss": 0,
    "hours-per-week": 48,
    "native-country": "United-States",
}
# request_data = {k.replace("-", "_"): v for k, v in request_data.items()}
%time r = requests.post(url="http://127.0.0.1:8000/prediction/", json=request_data,)
print(r.status_code)
print(r.json())

CPU times: user 4.6 ms, sys: 2.35 ms, total: 6.95 ms
Wall time: 25.4 ms
200
{'predictions': ['<=50K']}


In [ ]:
r = requests.get(url="http://127.0.0.1:8000/")
r

In [ ]:
r.json()